$$
R_x(\theta) = \begin{pmatrix} cos(\frac{\theta}{2}) & -isin(\frac{\theta}{2}) \\ -isin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{pmatrix} \\
R_y(\theta) = \begin{pmatrix} cos(\frac{\theta}{2}) & -sin(\frac{\theta}{2}) \\ sin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{pmatrix} \\
R_z(\theta) = \begin{pmatrix} e^{-i\frac{\theta}{2}} & 0 \\ 0 & e^{i\frac{\theta}{2}} \end{pmatrix} \\
$$


$$
S = \begin{pmatrix} 1 & 0 \\ 0 & i \end{pmatrix} \qquad
\propto \qquad
R_z(\frac{\pi}{2})
$$

$$
H = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} \qquad
\propto \qquad 
R_z(\pi)R_y(\frac{\pi}{2}) \qquad
\propto \qquad 
R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2}) \qquad
$$

$$
\begin{align}
R_y(\theta) &= SR_x(\theta)S^{\dagger}\\
         &= R_z(\frac{\pi}{2})R_x(\theta)R_z(-\frac{\pi}{2})
\end{align}
$$

Format of circuit input:
<center>
[[name_of_the_gate, [index_1, index_2]]]
</center>
only 2-qubit gates have index_2.

In [115]:
#example
import qiskit
from qiskit.circuit.library import XGate, YGate, ZGate, HGate, RXGate, RYGate, RZGate
import numpy as np

In [116]:
# Helper functions

BASIS_GATES = {'i', 'h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'cx', 'cz'}

#ng = no global phase

def get_param(gate):
    return gate.params[0]

def matrix(gates, precision=3):
    if len(gates) == 0:
        return 1
    return np.round(gates[-1].to_matrix().dot(matrix(gates[:-1])), precision)

def matrix_ng(gates, precision=3):
    if len(gates) == 0:
        return 1
    mat = np.round(gates[-1].to_matrix().dot(matrix(gates[:-1])), precision)
    for i in range(len(mat[0])):
        if mat[0][i] != 0:
            global_phase_angle = np.angle(mat[0][i])
            break
    global_phase = np.e**(global_phase_angle*1j)
    mat = mat/global_phase
    return np.round(mat, precision)

def equal(gates1, gates2):
    return (matrix(gates1) == matrix(gates2)).all()

def equal_ng(gates1, gates2):
    return (matrix_ng(gates1) == matrix_ng(gates2)).all()
    
def commute(gates1, gates2):
    m1 = matrix(gates1)
    m2 = matrix(gates2)
    return (m1.dot(m2) == m2.dot(m1)).all()

def commute_ng(gates1, gates2):
    cm1 = matrix_ng(gates1+gates2)
    cm2 = matrix_ng(gates2+gates1)
    return (cm1 == cm2).all()

def gate_to_str(gate):
    return gate.name if len(gate.params)==0 else f'{gate.name}({gate.params[0]/np.pi}pi)'

def equal_test(gates1, gates2):
    print(f'{[gate_to_str(g) for g in gates1]} vs {[gate_to_str(g) for g in gates2]}')
    print(f'equal: {equal(gates1, gates2)}')
    print(f'equal up to global phase: {equal_ng(gates1, gates2)}')
    print()

def commute_test(gates1, gates2):
    print(f'{[gate_to_str(g) for g in gates1]} vs {[gate_to_str(g) for g in gates2]}')
    print(f'commute: {commute(gates1, gates2)}')
    print(f'commute up to global phase: {commute_ng(gates1, gates2)}')
    print()

gates1 = [XGate(), ZGate()]
gates2 = [ZGate(), XGate()]
equal_test(gates1, gates2)
    
gates1 = [XGate()]
gates2 = [ZGate()]
commute_test(gates1, gates2)

gates1 = [XGate()]
gates2 = [RXGate(np.pi)]
equal_test(gates1, gates2)

gates1 = [HGate()]
gates2 = [RZGate(np.pi), RYGate(np.pi/2)]
equal_test(gates1, gates2)

['x', 'z'] vs ['z', 'x']
equal: False
equal up to global phase: True

['x'] vs ['z']
commute: False
commute up to global phase: True

['x'] vs ['rx(1.0pi)']
equal: False
equal up to global phase: True

['h'] vs ['rz(1.0pi)', 'ry(0.5pi)']


CircuitError: "to_matrix not defined for this <class 'qiskit.circuit.library.standard_gates.rz.RZGate'>"